## AI Inference on a local camera
This notebook is a simple example how to use DeGirum PySDK to do AI inference of a video stream from local camera.

This script works with the following inference options:

1. Run inference on DeGirum Cloud Platform;
2. Run inference on DeGirum AI Server deployed on a localhost or on some computer in your LAN or VPN;
3. Run inference on DeGirum ORCA accelerator directly installed on your computer.

To try different options, you just need to uncomment **one** of the lines in the code below.

You also need to specify your cloud API access token, cloud zoo URLs, and AI server hostname in [env.ini](env.ini) file, located in the same directory as this notebook.

**Access to camera is required to run this sample.**

The script needs either a web camera or local camera connected to the machine running this code. The camera index or URL needs to be specified either in the code below by assigning `camera_id` or in [env.ini](env.ini) file by defining `CAMERA_ID` variable and assigning `camera_id = None`.

#### Specify camera index here

In [ ]:
camera_id = None         # camera index or URL; 0 to use default local camera, None to take from env.ini file

#### Specify where do you want to run your inferences

In [ ]:
import degirum as dg # import DeGirum PySDK
import mytools

cloud_token = mytools.get_token() # get cloud API access token from env.ini file
cloud_zoo_url = mytools.get_cloud_zoo_url() # get cloud zoo URL from env.ini file

#
# Please UNCOMMENT only ONE of the following lines to specify where to run AI inference
#

# 1. Inference on the DeGirum Cloud Platform
zoo = dg.connect_model_zoo("dgcps://cs.degirum.com" + cloud_zoo_url, cloud_token)

# 2. Inference on DeGirum AI Server deployed on a localhost or on some computer in your LAN or VPN
# zoo = dg.connect_model_zoo((mytools.get_ai_server_hostname(), "https://cs.degirum.com" + cloud_zoo_url), cloud_token)

# 3. Inference on DeGirum ORCA accelerator installed on your computer
# zoo = dg.connect_model_zoo("https://cs.degirum.com" + cloud_zoo_url, cloud_token)


#### The rest of the cells below should run without any modifications

In [ ]:
# load AI model 'ssd_mobilenet_v2' for DeGirum Orca AI accelerator
# (change model name to "...n2x_cpu_1" to run it on CPU)
model = zoo.load_model("mobilenet_v2_ssd_coco--300x300_quant_n2x_orca_1")

# set model parameters
model.image_backend = 'opencv' # select OpenCV backend: needed to have overlay image in OpenCV format
model.input_numpy_colorspace = 'BGR'
model.overlay_show_probabilities = True # show probabilities on overlay image
model._model_parameters.InputImgFmt = ['JPEG']

In [ ]:
# AI prediction loop
# Press 'x' or 'q' to stop
with mytools.Display("AI Camera") as display, \
     mytools.open_video_stream(camera_id) as stream:
    
    for res in model.predict_batch(mytools.video_source(stream)):
        display.show(res.image_overlay)